In [1]:
import numpy as np
import pandas as pd
import squarify
from datetime import datetime
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings; warnings.filterwarnings(action='once')

Для выполнения данной лабораторной работы я взял данные с сайта [Faceit.com](https://www.faceit.com/ru/players/-Sunic). Это площадка, на которой можно играть в разные игры на рейтинг. Я провел около 2-х тысяч матчей в игре *CS:GO* на этой платформе за 2 года и решил собрать статистику **своих** матчей для анализа того, как я отыгрывал эти 2 года и почему не смог добиться тех задач, которые стояли передо мной в начале пути. 

In [2]:
data = pd.read_csv('PycharmProjects/seleniumProject/main/data.csv')

Т.к. данные на сайте прогружаются постепенно из-за JS скриптов, пришлось воспользоваться инструментом Selenium и написать код, который тратит примерно 5ч на сбор информации о каждом матче, т.к. каждый из них - это отдельная страница, к тому же не очень хотелось получить проверку на капчу от Гугла. Естественно, иногда возникали ошибки во время выполнения программы, но в итоге с n-ой попытки удалось собрать информацию о всех матчах в режиме 5v5, перевести информацию в нужный формат и в нужном стиле. Можно начинать анализ!   
  
*Код программы faceit.py:*
```python
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import time


with open('data.csv', 'r') as f:
    data = f.readlines()

try:
    opts = Options()
    opts.add_argument("user-agent=[user-agent string]")
    opts.add_argument("user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 "
                      "(KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36")
    driver = webdriver.Chrome(chrome_options=opts)
    driver.maximize_window()
    driver.get("https://www.faceit.com/ru/players/-Sunic/stats/csgo")
    time.sleep(10)
    tbody = driver.find_element_by_tag_name("tbody")
    trs = tbody.find_elements_by_tag_name("tr")

    # мотаем вниз пока все матчи не будут отображены
    while len(trs) < 1984:
        driver.execute_script("document.getElementById('main-container-height-wrapper').scrollBy(0, 3100)")
        trs = tbody.find_elements_by_tag_name("tr")
        time.sleep(.1)

    # получаем статистику каждого матча
    for tr in list(filter(lambda x: '5v5' in x.text, trs))[::-1][len(data) - 1:]:
        cdata = '-'.join(tr.find_elements_by_tag_name('td')[0].find_element_by_tag_name('span')
                         .get_attribute('uib-tooltip').split(',')[1].lstrip(' ').replace('Jan', '01')
                         .replace('Feb', '02').replace('Mar', '03').replace('Apr', '04').replace('May', '05')
                         .replace('Jun', '06').replace('Jul', '07').replace('Aug', '08').replace('Sep', '09')
                         .replace('Oct', '10').replace('Nov', '11').replace('Dec', '12').split(' ')[::-1]) + \
                ','+tr.text.split('\n')[1]+','+str(sum(int(x) for x in tr.text.split('\n')[2].split(' / '))) + \
                ','+tr.text.split('\n')[3].split('_')[1]+','
        tr.click()
        time.sleep(10)
finally:
    time.sleep(5)
    driver.quit()
```

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1960 entries, 0 to 1959
Data columns (total 12 columns):
Date        1960 non-null object
Result      1960 non-null object
Rounds      1960 non-null int64
Map         1960 non-null object
Position    1960 non-null int64
Kills       1960 non-null int64
Assists     1960 non-null int64
Deaths      1960 non-null int64
KR          1960 non-null float64
KD          1960 non-null float64
HS          1960 non-null int64
HSP         1960 non-null int64
dtypes: float64(2), int64(7), object(3)
memory usage: 183.8+ KB


Одними из главных показателей для анализа будут *Position* - место в рейтинге команды за матч (от 1 до 5), *KR* - кол-во убийств за раунд (в среднем за матч), *KD* - отношение кол-ва убийств к смертям за матч, *HSP* - процент хэдшотов из всех убийств (в КСГО нужно уметь убивать в голову)

In [4]:
data.head(5)

,Date,Result,Rounds,Map,Position,Kills,Assists,Deaths,KR,KD,HS,HSP
0,2017-11-04,WIN,36,cache,3,24,6,28,0.67,0.86,10,42
1,2017-11-04,LOSS,19,cache,4,11,3,15,0.58,0.73,3,27
2,2017-11-04,WIN,36,overpass,5,23,11,26,0.64,0.88,7,30
3,2017-11-05,WIN,25,train,3,16,3,13,0.64,1.23,2,12
4,2017-11-05,LOSS,35,mirage,5,20,4,25,0.57,0.80,2,10


In [5]:
data.describe()

,Rounds,Position,Kills,Assists,Deaths,KR,KD,HS,HSP
count,1960.000000,1960.000000,1960.000000,1960.000000,1960.000000,1960.000000,1960.000000,1960.000000,1960.000000
mean,26.037245,3.276531,17.060204,3.753571,17.936224,0.655485,1.049184,6.647449,39.180612
std,5.742575,1.331696,6.409972,2.249153,5.293771,0.208534,0.600659,3.275303,14.203546
min,6.000000,1.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000
25%,22.000000,2.000000,13.000000,2.000000,15.000000,0.520000,0.700000,4.000000,30.000000
50%,26.000000,3.000000,17.000000,3.000000,19.000000,0.640000,0.940000,6.000000,39.000000
75%,29.000000,4.000000,21.000000,5.000000,21.000000,0.780000,1.240000,9.000000,47.000000
max,58.000000,5.000000,48.000000,13.000000,39.000000,1.640000,7.500000,25.000000,100.000000


# Подготовка данных

In [ ]:
data.isnull().sum()

Пропусков в данных нет

In [ ]:
data = data.replace("WIN", 1)
data = data.replace("LOSS", 0)

Я заменил значения Result для удобства на цифры 1 и 0

In [ ]:
data.KR.describe()

In [ ]:
np.average(data.KD)

In [ ]:
np.average(data.HSP)

Хорошие новости в том, что 3 важнейших показателя индивидуального скилла на хорошем уровне для любительской игры (по моему субъективному взгляду, полученному за 2 года опыта), но не на достаточном, чтобы находиться выше.

In [13]:
data.head(10)

,Date,Result,Rounds,Map,Position,Kills,Assists,Deaths,KR,KD,HS,HSP
0,2017-11-04,WIN,36,cache,3,24,6,28,0.67,0.86,10,42
1,2017-11-04,LOSS,19,cache,4,11,3,15,0.58,0.73,3,27
2,2017-11-04,WIN,36,overpass,5,23,11,26,0.64,0.88,7,30
3,2017-11-05,WIN,25,train,3,16,3,13,0.64,1.23,2,12
4,2017-11-05,LOSS,35,mirage,5,20,4,25,0.57,0.80,2,10
5,2017-11-05,WIN,18,inferno,4,15,2,7,0.83,2.14,8,53
6,2017-11-05,LOSS,22,mirage,3,13,1,17,0.59,0.76,7,54
7,2017-11-06,LOSS,18,dust2,2,6,1,16,0.33,0.38,2,33
8,2017-11-06,WIN,30,dust2,5,15,4,19,0.50,0.79,3,20
9,2017-11-06,LOSS,29,mirage,2,24,2,23,0.83,1.04,7,29


In [15]:
data['Period'] = 0
for i, r in data.iterrows():
    data.at[i, 'Period'] = (datetime.strptime(str(r['Date']), '%Y-%m-%d') - datetime.strptime('2017-11-04', '%Y-%m-%d'))\
                           .days // 90 + 1

In [16]:
low_periods = list(data.groupby('Period').size().reset_index(name="count")\
                       .loc[data.groupby('Period').size().reset_index(name="count")["count"] > 10].Period)

In [17]:
data = data.loc[data['Period'].isin(low_periods)]

Чтобы удобнее было работать с датами, я добавил колонку "Period" в data, в каждом из которых результаты 90 матчей за этот период (от первого до последнего) и выбрал только те периоды, в которых матчей было хотя бы больше 10.

In [18]:
data.Map.unique()

array(['cache', 'overpass', 'train', 'mirage', 'inferno', 'dust2',
       'cbble', 'nuke', 'vertigo'], dtype=object)

In [19]:
data = data.drop(data[data.Map == 'vertigo'].index)

Для чистоты статистики я решил убрать данные по карте "vertigo", потому что я ней я играл всего 1 раз

In [20]:
data.count()[0]

1948

In [22]:
print(',Date,Result,Rounds,Map,Position,Kills,Assists,Deaths,KR,KD,HS,HSP')
for i, r in data.iterrows():
    r['Date'] = datetime.strptime(r['Date'], '%Y-%m-%d').strftime('%d.%m.%y')
    print(f"{i+1},{r['Date']},{r['Result']},{r['Rounds']},{r['Map']},{r['Position']},{r['Kills']},{r['Assists']},{r['Deaths']},{r['KR']},{r['KD']},{r['HS']},{r['HSP']}")

,Date,Result,Rounds,Map,Position,Kills,Assists,Deaths,KR,KD,HS,HSP
1,04.11.17,WIN,36,cache,3,24,6,28,0.67,0.86,10,42
2,04.11.17,LOSS,19,cache,4,11,3,15,0.58,0.73,3,27
3,04.11.17,WIN,36,overpass,5,23,11,26,0.64,0.88,7,30
4,05.11.17,WIN,25,train,3,16,3,13,0.64,1.23,2,12
5,05.11.17,LOSS,35,mirage,5,20,4,25,0.57,0.8,2,10
6,05.11.17,WIN,18,inferno,4,15,2,7,0.83,2.14,8,53
7,05.11.17,LOSS,22,mirage,3,13,1,17,0.59,0.76,7,54
8,06.11.17,LOSS,18,dust2,2,6,1,16,0.33,0.38,2,33
9,06.11.17,WIN,30,dust2,5,15,4,19,0.5,0.79,3,20
10,06.11.17,LOSS,29,mirage,2,24,2,23,0.83,1.04,7,29
11,06.11.17,WIN,27,cache,4,18,3,15,0.67,1.2,5,28
12,07.11.17,WIN,27,mirage,4,13,6,19,0.48,0.68,4,31
13,07.11.17,LOSS,26,mirage,5,8,2,21,0.31,0.38,1,12
14,08.11.17,LOSS,41,inferno,5,22,5,28,0.54,0.79,2,9
15,08.11.17,LOSS,25,mirage,3,15,2,21,0.6,0.71,3,20
16,08.11.17,LOSS,26,mirage,3,18,4,23,0.69,0.78,12,67
17,08.11.17,LOSS,24,cache,3,12,4,22,0.5,0.55,3,25
18,08.11.17,WIN,23,overpass,5,13,5,16,0.57,0.81,5,38
19,08.11.17,LOSS,21,c

816,18.06.18,WIN,29,mirage,3,20,5,20,0.69,1.0,7,35
817,18.06.18,WIN,30,cache,5,14,5,21,0.47,0.67,4,29
818,19.06.18,WIN,20,inferno,5,10,4,10,0.5,1.0,4,40
819,19.06.18,WIN,10,inferno,4,7,1,4,0.7,1.75,1,14
820,19.06.18,LOSS,34,inferno,5,17,3,26,0.5,0.65,3,18
821,20.06.18,WIN,21,mirage,5,8,6,10,0.38,0.8,1,12
822,28.06.18,LOSS,24,mirage,2,14,3,17,0.58,0.82,7,50
823,28.06.18,LOSS,23,nuke,5,10,4,18,0.43,0.56,4,40
824,28.06.18,LOSS,25,cbble,2,16,4,18,0.64,0.89,5,31
825,29.06.18,WIN,30,cache,5,12,10,23,0.4,0.52,3,25
826,30.06.18,LOSS,28,cache,5,14,3,21,0.5,0.67,6,43
827,30.06.18,WIN,21,mirage,4,16,5,12,0.76,1.33,2,12
828,30.06.18,LOSS,29,overpass,5,11,7,24,0.38,0.46,2,18
829,30.06.18,WIN,27,mirage,4,20,4,17,0.74,1.18,10,50
830,30.06.18,WIN,21,mirage,1,23,3,11,1.1,2.09,7,30
831,30.06.18,LOSS,29,cache,1,22,8,24,0.76,0.92,7,32
832,01.07.18,LOSS,36,mirage,1,33,6,27,0.92,1.22,13,39
833,01.07.18,LOSS,30,cache,3,22,4,24,0.73,0.92,8,36
834,01.07.18,WIN,35,mirage,5,22,4,26,0.63,0.85,10,45
835,01.07.18,W

1708,06.02.19,LOSS,19,mirage,5,6,3,17,0.32,0.35,2,33
1709,06.02.19,LOSS,30,inferno,1,29,3,28,0.97,1.04,15,52
1710,06.02.19,WIN,42,cache,2,38,9,31,0.9,1.23,11,29
1711,06.02.19,WIN,34,mirage,3,23,4,21,0.68,1.1,15,65
1712,06.02.19,WIN,26,mirage,3,20,3,15,0.77,1.33,12,60
1713,06.02.19,WIN,21,inferno,4,16,2,13,0.76,1.23,6,38
1714,07.02.19,WIN,24,mirage,5,8,2,18,0.33,0.44,2,25
1715,07.02.19,LOSS,24,mirage,3,13,2,21,0.54,0.62,5,38
1716,07.02.19,WIN,24,overpass,5,14,5,14,0.58,1.0,4,29
1717,07.02.19,WIN,30,mirage,4,16,2,20,0.53,0.8,6,38
1718,07.02.19,LOSS,24,overpass,5,10,2,19,0.42,0.53,4,40
1719,07.02.19,LOSS,26,inferno,1,21,4,19,0.81,1.11,8,38
1720,07.02.19,WIN,35,dust2,5,14,5,24,0.4,0.58,7,50
1721,08.02.19,WIN,22,mirage,5,12,4,12,0.55,1.0,4,33
1722,08.02.19,WIN,27,cache,4,21,4,15,0.78,1.4,8,38
1723,08.02.19,WIN,24,mirage,5,11,2,13,0.46,0.85,5,45
1724,08.02.19,WIN,18,cache,5,8,5,7,0.44,1.14,2,25
1725,09.02.19,WIN,29,mirage,5,18,5,21,0.62,0.86,8,44
1726,09.02.19,LOSS,20,dust2,3,8,2,15,0.4,0.53

In [ ]:
from math import floor
from collections import OrderedDict
newdata = OrderedDict()
for i in range(39):
    newdata[f'[{i * 50 + 1}-{i * 50 + 50}]'] = []
for i, r in data.iterrows():
    newdata[f'[{floor(i / 50) * 50 + 1}-{floor(i / 50) * 50 + 50}]'].append({'KD': r['KD'], 'Days': r['Days'], 'KR': r['KR']})
print('Mathces,Days,KD,KR')
for k, v in newdata.items():
    print(f"{k},{(v[-1]['Days'] - v[0]['Days']) / 100},{sum(x['KD'] for x in v) / len(v)},{sum(x['KR'] for x in v) / len(v)}")


В итоге получилось 1948 матчей, по которым я буду собирать разного рода статистику и анализировать. Приступим!

In [ ]:
data.to_csv('PycharmProjects/seleniumProject/main/data1.csv')

In [ ]:
data.Assists.sum()

# Описательная статистика

In [ ]:
def get_result():
    results = ["Поражение", "Победа"]
    for res in results:
        yield res
result = get_result()

fig, ax = plt.subplots(figsize=(15, 15))
wedges, texts, autotexts = ax.pie(data.groupby('Result').size(),
                                  autopct=lambda pct: f"{next(result)}\n{pct:.1f}%",
                                  textprops=dict(color="w"), wedgeprops=dict(width=0.5))
plt.setp(autotexts, size=20, weight="bold")
ax.set_title("Результаты всех матчей за все время", fontsize=20)
plt.show()

результаты матчей примерно равны по кол-ву, но все же побед немного больше и это приятно :)

In [ ]:
def bins_labels(bins, **kwargs):
    bin_w = (max(bins) - min(bins)) / (len(bins) - 1)
    plt.xticks(np.arange(min(bins)+bin_w/2, max(bins), bin_w), bins, **kwargs)
    plt.xlim(bins[0], bins[-1])

plt.figure(figsize=(16,5))
plt.hist(data.Period, bins=range(1, 10))
bins_labels(range(1, 10), fontsize=18)
plt.show()

Здесь показано распределение кол-ва проведенных матчей по периодам. Как можно заметить, наибольшая активность была в течение первых полутора лет, причем со 2-го периода по 4-й (примерно февраль 2018 - октябрь 2018) я проводил наибольшее кол-во время за игрой, примерно по 4-5 матчей в день в среднем.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 15))
colors = ["purple", "red", "blue", "orange", "green"]
explode = (0, 0, 0, 0.1, 0)
wedges, texts, autotexts = ax.pie(data.groupby('Position').size(),
                                  colors=colors,
                                  explode=explode,
                                  shadow=True,
                                  autopct=lambda pct: f"{pct:.1f}%",
                                  textprops=dict(color="w"))
ax.legend(wedges, sorted(data.Position.unique()),
          title="Позиция",
          loc="center left",
          bbox_to_anchor=(1, 0, 0.5, 1),
          fontsize=18)
plt.setp(autotexts, size=18, weight="bold")
ax.set_title("Позиции в командном зачете за все время", fontsize=18)
plt.show()

А по результатам выходит, что большинство матчей я заканчивал на последних местах в командом зачете вне зависимости от результата.

In [ ]:
data.hist('Position', by='Map', figsize=[15, 15]);

Гистограммы по картам тоже не дают позитив. Средняя позиция 3 выбивается лишь на картах cobblestone, на которой было не так много матчей, и на карте cache. В остальном те же 4-5 места в среднем.

In [ ]:
data_position_counts = data.groupby(['Position', 'Period']).size().reset_index(name='counts')

# Draw Stripplot
fig, ax = plt.subplots(figsize=(16, 6), dpi=80)    
sns.scatterplot(data_position_counts.Period, data_position_counts.Position, size=data_position_counts.counts*2, ax=ax)

# Decorations
plt.title('Количество позиций в зависимости от периода', fontsize=20)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
sns.lineplot(x=data.Period, y=data.HSP).set_title(f'Средний % HS')
sns.lineplot(x=data.Period, y=data.drop(data[data.Kills < 15].index).HSP, color='gold')

Интересно было посмотреть, как % попадания в голову зависит от кол-ва убийств в целом за матче. Как видно, желтый график, который включает в себя только данные за матчи, в которых было не менее 15 убийств, ниже синего графика, в котором запечатленены все данные. Это логично. Ведь либо игрок, который делает много убийств за игру, делает их из-за сверхконцентрации в момент матча и тут не играет роль сделан килл в голову или нет, либо много убийств получается засчет соперников с низким уровнем здоровья, поэтому пули в тело им достаточно, чтобы убить.

# Гипотезы

### **Гипотеза 1.** Чем больше сыграно матчей за определенный период - тем хуже индивидуальные скилы игрока

In [ ]:
data_mean = data.groupby(['Period']).mean()
data_mean['count'] = 0
for i, r in data_mean.iterrows():
    data_mean.at[i, 'count'] = data.groupby(['Period']).size()[i]
data_mean.head()
data_mean.to_csv('PycharmProjects/seleniumProject/main/data_mean.csv')

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,6))
for i, t in enumerate([('Position', data_mean.Position), ('KR', data_mean.KR), ('KD', data_mean.KD), ('HSP', data_mean.HSP)]):
    ax = pd.DataFrame({'x': range(1, len(data.Period.unique())+1), t[0]: t[1]})\
           .plot('x', t[0], ax=axes[i // 2, i % 2], color='blue')
    ax.set_xlabel('')

**Гипотеза опровергнута**, потому что, как видно из графиков выше, сильного спада статистики в период максимальной активности (периоды 2-4) не обнаружена, за исключением *kd*, который продолжал опускаться и после выбранных периодов, а средний процент хэдшотов наоборот рос на протяжении всех периодов. 

### **Гипотеза 2:** под конец дня индивидуальный скилл и концентрация игрока падает

In [ ]:
dd = data
for d in dd.Date.unique():
    if len(dd.groupby(['Date']).get_group(d)) < 4:
        dd = dd.drop(dd[dd.Date == d].index)

dd = dd.groupby('Date')['KR', 'HSP'].apply(lambda x: [list(x['KR']), list(x['HSP'])]).reset_index(name="Stat")
dd.colums = ['Date', 'KR', 'HSP']
kr_start, kr_end, hsp_start, hsp_end = [], [], [], []
for i, r in dd.iterrows():
    kr_start.append(sum(r['Stat'][0][:2]) / 2)
    kr_end.append(sum(r['Stat'][0][-2:]) / 2)
    hsp_start.append(sum(r['Stat'][1][:2]) / 2)
    hsp_end.append(sum(r['Stat'][1][-2:]) / 2)

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
colors = ['red', 'blue']
for i, t in enumerate([('kr_start', kr_start), ('kr_end', kr_end),\
                       ('hsp_start', hsp_start), ('hsp_end', hsp_end)]):
    ax = pd.DataFrame({'x': range(1, len(kr_start)+1), t[0]: t[1]}).plot('x', t[0], ax=axes[i // 2], color=colors[i % 2])
    ax.set_xlabel('')

Я выбрал только дни, в которых было хотя бы 4 матча и сравнил показатели за первые 2 матча и последние 2 матча. Прадва, такие графики очень сложные для анализа, поэтому я усреднил их значения по 12

In [ ]:
import math
kr_start = list(sum(kr_start[i*12:(i+1)*12]) / len(kr_start[i*12:(i+1)*12]) for i in range(0, math.ceil(len(kr_start) / 12)))
kr_end = list(sum(kr_end[i*12:(i+1)*12]) / len(kr_end[i*12:(i+1)*12]) for i in range(0, math.ceil(len(kr_end) / 12)))
hsp_start = list(sum(hsp_start[i*12:(i+1)*12]) / len(hsp_start[i*12:(i+1)*12]) for i in range(0, math.ceil(len(hsp_start) / 12)))
hsp_end = list(sum(hsp_end[i*12:(i+1)*12]) / len(hsp_end[i*12:(i+1)*12]) for i in range(0, math.ceil(len(hsp_end) / 12)))
kr = list(kr_start[i] - kr_end[i] for i in range(len(kr_start)))
hsp = list(hsp_start[i] - hsp_end[i] for i in range(len(hsp_start)))
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(16, 6))
for i, t in enumerate([('kr_start', kr_start), ('kr_end', kr_end),\
                       ('hsp_start', hsp_start), ('hsp_end', hsp_end)]):
    ax = pd.DataFrame({'x': range(1, len(kr_start)+1), t[0]: t[1]}).plot('x', t[0], ax=axes[i // 2], color=colors[i % 2])
    ax.set_xlabel('')

В действительности имеем, что статистика по убийствам за раунд под конец дня лишь вырастала, а средний процент попадания в голову хоть и выше иногда в начале дня, но в среднем можно считать примерно равным. **Гипотеза опровергнута.**

### Гипотеза 3: длительность матча (в раундах) обратнопропорциональна индивидуальному результату игрока

In [ ]:
rounds_bins = [0, 24, 30, 59]
data['Duration'] = pd.cut(data.Rounds, rounds_bins)
data_durs = data.groupby('Duration').size().reset_index(name='Durs')

labels = data_durs.apply(lambda x: str(x[0]).replace("(0, 24]", "Проходная катка")\
                         .replace("(24, 30]", "Нелегкая катка")\
                         .replace("(30, 59]", "Ну и пот") + "\n (" + str(x[1]) + ")", axis=1)
plt.figure(figsize=(16,2), dpi= 80)
squarify.plot(sizes=data_durs['Durs'].values.tolist(), label=labels,
              color=["purple", "orange", "green"], alpha=.8, text_kwargs=dict(color="w", fontsize="15", weight="bold"))

plt.title('Длительность матчей (в раундах)')
plt.axis('off')
plt.show()

In [ ]:
data_durs = data.groupby(["Period", "Duration"]).mean().reset_index()

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(16,8))
colors = ["red", "blue", "orange"]

for i, t in enumerate([('Position', data_durs.Position), ('KR', data_durs.KR), ('KR', data_durs.KD), ('HSP', data_durs.HSP)]):
    for j, dur in enumerate(data_durs.Duration.unique()):
        ax = pd.DataFrame({'x': range(1, len(data.Period.unique())+1), t[0]: t[1].loc[data_durs['Duration'] == dur]})\
               .plot('x', t[0], ax=axes[i // 2, i % 2], color=colors[j], label=f'{t[0]} {dur}')
        ax.set_xlabel('')

Графики *kd* и *hsp* явно отображают то, что скилл в матчах с кол-вом раундов менее 24 в среднем лучше, чем в матчах с любым другим кол-вом матчей. Тем не менее, графики *position* и *kr* не такие очевидные. Скорее график *position* показывает, что в сложных матчах ситуация стабильнее, чем в других матчах, а график *kr* что скилл в легких матчах со временем падал, зато в потных катках все чаще был высокого уровня. В общем и целом можно считать, что **гипотеза частично доказана.**

** Вывод:** в результате выполнения данной лабораторной работы я узнал немало новой информации, а главное посмотрел на себя с другой стороны. Да, изначально я хотел проанализировать все свои матчи и понять почему я смог развиваться так, как хотел в самом начале истории, но в анализе не было учтено множество факторов, которые нельзя привязать к матчам, которые никто не будет записывать (настроение, время тренировки ДО матча, играю я в команде с друзьями или случайными игроками, и самое главное - важнейший показатель уровня игры, который не учитывает *Faceit* по неизвестной мне причине - AVG). В любом случае, это был интересный опыт. Для меня это точно лучше, чем анализировать кучу непонятных данных и пытаться что-то спрогнозировать, не понимая смысла этих данных. Здесь же я опроверг, или частично опроверг 3 гипотезы, которые были со мной последний год активности и в которых я раннее был уверен, и это очень интересно!